In [0]:
from pyspark.sql.functions import *

In [0]:
# # Read source data from CSV
# source_df = spark.read.option("header", True).option("inferschema", True).csv("/mnt/input-data/employees.csv");


# source_df = source_df.filter(col("EMPLOYEE_ID")> 200)

# source_df.display()




In [0]:

# source_df.write.format("delta")\
#     .mode("overwrite")\
#         .save("/tmp/employees")

In [0]:
# %sql
# create table if not exists employees
# using delta;

In [0]:
df22 = spark.read.format("delta").load("/tmp/employees")
df22.display()

In [0]:
# Read source data from CSV
source_df = spark.read.option("header", True).option("inferschema", True).csv("/mnt/input-data/employees.csv")

# source_df.display()
# Read the existing target Delta table as a DataFrame
target_df = spark.read.format("delta").load("/tmp/employees")

source_df.display()
target_df.display()


In [0]:
# Join source and target on employee_id
joined_df = source_df.alias("src").join(
    target_df.alias("tgt"),
    on="EMPLOYEE_ID",
    how="left"
)

# Find records that need to be updated (any column changed)
updates_df = joined_df.filter(
    (joined_df["src.FIRST_NAME"] != joined_df["tgt.FIRST_NAME"]) |
    (joined_df["src.LAST_NAME"] != joined_df["tgt.LAST_NAME"]) |
    (joined_df["src.DEPARTMENT_ID"] != joined_df["tgt.DEPARTMENT_ID"]) |
    (joined_df["src.SALARY"] != joined_df["tgt.SALARY"])
).select("src.*")

# Find records that are new (not present in target)
new_records_df = source_df.join(target_df, on="EMPLOYEE_ID", how="left_anti")




In [0]:
# Remove old versions of updated records from the target
unchanged_df = target_df.join(updates_df, on="EMPLOYEE_ID", how="left_anti")

# Combine unchanged + updated + new records
final_df = unchanged_df.union(updates_df).union(new_records_df)

# Overwrite the target Delta table
final_df.write.format("delta").mode("overwrite").save("/tmp/employees")


In [0]:
df22 = spark.read.format("delta").load("/tmp/employees")
df22.display()